In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    # "FASTX",
    # "StatsBase",
    # "Distributions",
    # "StatsPlots",
    # "Random",
    # "Dates",
    # "DataFrames",
    # "BioSequences",
    # "Conda",
    # "Downloads"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
bams = sort(filter(x -> occursin(r"\.sorted\.bam$", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
software_dir = mkpath("$(homedir())/software")
freebayes = joinpath(software_dir, "bin", "freebayes-1.3.6-linux-amd64-static")
if !isfile(freebayes)
    import Downloads
    # url = "https://github.com/freebayes/freebayes/archive/refs/tags/v1.3.7.zip"
    # url = "https://github.com/freebayes/freebayes/archive/refs/tags/v1.3.7.tar.gz"
    url = "https://github.com/freebayes/freebayes/releases/download/v1.3.6/freebayes-1.3.6-linux-amd64-static.gz"
    output = joinpath(software_dir, "bin", basename(url))
    Downloads.download(url, output)
    # run(`unzip $(output) -d $(software_dir)`)
    # tar $(output) -d $(software_dir)`)
    # run(`tar --extract --file=$(output) --directory=$(software_dir)`)
    # rm(output)
    run(`gzip -d $(output)`)
end
run(`chmod +x $(freebayes)`)

In [ ]:
# bams = [
#     # "3NEzu1DmBy.fna.normalized.vcf.fna.badread.1000x.filtlong.fq.gz.minimap2.sorted.bam",
#     # "GCF_024820135.1.fna.normalized.vcf.fna.badread.1000x.filtlong.fq.gz.minimap2.sorted.bam",
#     # "19tEIXfKKdzr.fna.normalized.vcf.fna.badread.1000x.filtlong.fq.gz.minimap2.sorted.bam",
#     "PyN5Z2YLzh7c97.fna.normalized.vcf.fna.badread.1000x.filtlong.fq.gz.minimap2.sorted.bam",
#     "GCF_023169545.1.fna.normalized.vcf.fna.badread.1000x.filtlong.fq.gz.minimap2.sorted.bam"
# ]

# bams = joinpath.(genome_dir, bams)

In [ ]:
threads = 4
for bam in bams
    outfile = bam * ".freebayes.vcf"
    fasta_reference = replace(bam, r"\.normalized\.vcf\.fna.*" => "")
    # if !isfile(outfile) || (filesize(outfile) == 0)
    # @info "$(outfile) not present, generating..."        
    cmd = 
    """
    $(freebayes) --ploidy 1 --use-best-n-alleles 1 --fasta-reference $(fasta_reference) --bam $(bam) --vcf $(outfile)
    """
    Mycelia.nersc_sbatch(
        job_name = "$(basename(outfile))",
        mail_user = "cameron.prybol@gmail.com",
        logdir = mkpath("$(homedir())/workspace/slurmlogs"),
        qos = "shared",
        cpus_per_task= threads,
        cmd = cmd)
    # else
    #     @info "$(outfile) already present..."
    # end
end